In [92]:
import requests

import boto3
import mlflow
from mlflow import pyfunc as ml_pyfunc
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from pickle import dump

In [93]:
########################################################
### Import Dataset
########################################################


In [97]:
## Import dataset
leads_dataset = pd.read_csv('data/Data.csv')
leads_dataset.columns = map(str.lower, leads_dataset.columns)

In [99]:
########################################################
### Clean and Prepare Data
########################################################
print(leads_dataset.columns)
print(leads_dataset.head())

Index(['unnamed: 0', 'tipo de ruc', 'estado civil', 'cantidad de cuentas',
       'saldo neto', 'tendencia', 'rubro', 'morosidad', 'historial crediticio',
       'antigüedad', 'calificación crediticia'],
      dtype='object')
   unnamed: 0 tipo de ruc estado civil  cantidad de cuentas   saldo neto  \
0           0     Natural      Soltero                    0  -482.146987   
1           1     Natural        Viudo                    5   191.853013   
2           2    Juridica   Divorciado                    2  1049.853013   
3           3    Juridica   Divorciado                    1  -177.146987   
4           4    Juridica       Casado                    2   945.853013   

   tendencia      rubro morosidad historial crediticio antigüedad  \
0        0.0      Otros  Riesgoso                 Bajo         No   
1        2.5      Otros  Riesgoso               Normal         No   
2        2.0  Servicios    Ligero                Medio         No   
3        1.0         TI  Riesgoso        

In [108]:
# Create data pre-processing steps before plugging into model
leads_categorical_columns = ['tipo de ruc',
                             'estado civil',
                             'rubro',
                             'morosidad',
                             'historial crediticio',
                            'antigüedad']

leads_numeric_columns = ['cantidad de cuentas',
                         'saldo neto',
                        'tendencia']

leads_response_columns = ['calificación crediticia']

In [109]:
#split data for training, remove extras

leads_x = leads_dataset.drop(leads_response_columns, axis=1)
leads_y = leads_dataset[leads_response_columns]

leads_x_train, leads_x_test, leads_y_train, leads_y_test = train_test_split(leads_x,
                                                                            leads_y,
                                                                            train_size=0.7,
                                                                            test_size=0.3,
                                                                            random_state=5050)

In [110]:
scaler = StandardScaler()
scaler = scaler.fit(leads_x_train[leads_numeric_columns])
dump(scaler, open('scaler.pkl', 'wb'))

In [111]:
def pre_process_leads_data(df,
                           numeric_columns,
                           categorical_columns,
                           fitted_scaler,
                           train_df_columns = None):
    ## create new df with selected columns
    df.columns = map(str.lower, df.columns)
    _df = df[set(numeric_columns + categorical_columns)].copy()
    
    
    ## scale the numeric columns with the pre-built scaler
    _df[numeric_columns] = fitted_scaler.transform(_df[numeric_columns])
         
    # First, make categorical text lowercase
    _df[categorical_columns] = _df[categorical_columns].apply(lambda x: x.str.lower())
    # Next, create one-hot-encoded variables, add to dataframe, drop old columns
    _df_dummies = pd.get_dummies(_df[categorical_columns], drop_first=True)
    _df = pd.concat([_df, _df_dummies], axis=1)
    _df.drop(categorical_columns, axis=1, inplace = True)

    if train_df_columns:
        _df = _df.reindex(columns=train_df_columns, fill_value=0)

    return _df

In [112]:
#print(leads_x_train.iloc[0])
leads_x_train_clean = pre_process_leads_data(df = leads_x_train,
                                            numeric_columns = leads_numeric_columns,
                                            categorical_columns = leads_categorical_columns,
                                            fitted_scaler = scaler)

leads_x_test_clean = pre_process_leads_data(df = leads_x_test,
                                           numeric_columns = leads_numeric_columns,
                                           categorical_columns = leads_categorical_columns,
                                           fitted_scaler = scaler,
                                           train_df_columns = leads_x_train_clean.columns.tolist())

saldo neto             -482.146987
antigüedad                      No
cantidad de cuentas              0
rubro                        Otros
estado civil            Divorciado
morosidad                 Riesgoso
tendencia                      0.0
tipo de ruc                Natural
historial crediticio        Normal
Name: 1208, dtype: object
saldo neto             -482.146987
antigüedad                      No
cantidad de cuentas              0
rubro                        Otros
estado civil               Soltero
morosidad                 Riesgoso
tendencia                      0.0
tipo de ruc                Natural
historial crediticio        Normal
Name: 4978, dtype: object


C:\Users\User\AppData\Local\Temp\ipykernel_14436\3539428584.py:8: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  _df = df[set(numeric_columns + categorical_columns)].copy()
C:\Users\User\AppData\Local\Temp\ipykernel_14436\3539428584.py:8: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  _df = df[set(numeric_columns + categorical_columns)].copy()


In [113]:
########################################################
### Train and Evaluate Model
########################################################

In [114]:
## Train the random forest model
num_estimators = 100
min_samples = 4

rf = RandomForestClassifier(n_estimators=num_estimators,
                            min_samples_split=min_samples)
rf.fit(leads_x_train_clean, leads_y_train.values.ravel())

RandomForestClassifier(min_samples_split=4)

In [115]:
leads_y_test_predicted = rf.predict(leads_x_test_clean)


accuracy = metrics.accuracy_score(leads_y_test, leads_y_test_predicted)
auc_score = metrics.roc_auc_score(leads_y_test, leads_y_test_predicted)

print(accuracy)
print(auc_score)

saldo neto                    -0.885623
cantidad de cuentas           -0.654124
tendencia                     -1.084492
tipo de ruc_natural            1.000000
estado civil_divorciado        0.000000
estado civil_soltero           1.000000
estado civil_viudo             0.000000
rubro_otros                    1.000000
rubro_servicios                0.000000
rubro_ti                       0.000000
morosidad_ligero               0.000000
morosidad_normal               0.000000
morosidad_riesgoso             1.000000
historial crediticio_bajo      0.000000
historial crediticio_medio     0.000000
historial crediticio_normal    1.000000
historial crediticio_riesgo    0.000000
Name: 4978, dtype: float64
0.8315557190143436
0.8166613537165819


In [116]:

import joblib

joblib.dump(rf, 'classifier.pkl')
########################################################
### MLflow and environment setup
########################################################

['classifier.pkl']

In [117]:
# connect to MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("LeadScoringProcessed") # creates an experiment if it doesn't exist

MlflowException: API request to http://localhost:5000/api/2.0/mlflow/experiments/list failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/list?view_type=ALL (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000018E8692C4C0>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

In [22]:
# define specific python and package versions for environment
mlflow_conda_env = {
 'name': 'mlflow-env',
 'channels': ['defaults'],
 'dependencies': ['python=3.8.13', {'pip': ['mlflow==1.24.0','scikit-learn==1.0.2','cloudpickle==2.0.0']}]
}

In [23]:
########################################################
### Define Model
########################################################

In [24]:
class leadsModel(mlflow.pyfunc.PythonModel):
   
    ## defining objects needed for leadsModel prediction. 
    def __init__(self,
                 train_df_columns,
                 model,
                 leads_categorical_columns,
                 leads_numeric_columns,
                 fitted_scaler,
                 pre_process_leads_data):
        
        ## Setting up all needed objects
        self.train_df_columns = train_df_columns
        self.model = model
        self.leads_categorical_columns = leads_categorical_columns
        self.leads_numeric_columns = leads_numeric_columns
        self.fitted_scaler = fitted_scaler
        self.pre_process_leads_data = pre_process_leads_data
    
    ## define function with processing and feeding data into prediction at the end
    def predict(self,context,model_input):
        
        # make sure all inputted columns are lowercase
        model_input.columns = map(str.lower, model_input.columns)
        
        # run inputted dataset through our processing function
        # note: we are excluding the response columns here since not needed for deploy
        model_input_processed = self.pre_process_leads_data(
                                   df = model_input,
                                   numeric_columns = self.leads_numeric_columns,
                                   categorical_columns = self.leads_categorical_columns,
                                   fitted_scaler = self.fitted_scaler,
                                   train_df_columns = self.train_df_columns)       
        
        # finally input the cleaned/adjusted dataset into our model for prediction
        return self.model.predict(model_input_processed)

In [25]:
# Testing the prediction class before pushing to MLflow
m = leadsModel(train_df_columns = leads_x_train_clean.columns.tolist(),
                                  model = rf,
                                  leads_categorical_columns = leads_categorical_columns,
                                  leads_numeric_columns = leads_numeric_columns,
                                  fitted_scaler = scaler,
                                  pre_process_leads_data = pre_process_leads_data)
model_input = leads_x.head(1)
model_output = m.predict(None,model_input)
print(model_output)

[0]


C:\Users\User\AppData\Local\Temp\ipykernel_14436\2053193086.py:8: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  _df = df[set(numeric_columns + categorical_columns)].copy()


In [ ]:
########################################################
### Log Model to MLflow
########################################################

In [27]:
# start mlflow run, log parameters, metrics, and the model
with mlflow.start_run(run_name="Leads Model with Processing") as run:
    # log the parameters that we defined for the model training
    mlflow.log_param("num_estimators", num_estimators)
    mlflow.log_param("min_samples", min_samples)
    
    # log the performance metrics that we calculated earlier
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("auc_score", auc_score)
    
    # log model with all objects referenced in the leadsModel class
    ml_pyfunc.log_model(
        artifact_path = "leads_pyfunc",
        python_model = leadsModel(train_df_columns = leads_x_train_clean.columns.tolist(),
                                  model = rf,
                                  leads_categorical_columns = leads_categorical_columns,
                                  leads_numeric_columns = leads_numeric_columns,
                                  fitted_scaler = scaler,
                                  pre_process_leads_data = pre_process_leads_data
                                 ),
        conda_env = mlflow_conda_env
    )
    
    # save run_id and experiment_id for deployment
    run_id = run.info.run_uuid
    experiment_id = run.info.experiment_id
    
    # end the mlflow run!
    mlflow.end_run()

In [ ]:
########################################################
### Test Local Deployment
########################################################

In [35]:
# Run this command in the same directory as MLflow to kick-off a local sagemaker build

sagemaker_local_command = 'mlflow sagemaker run-local -m ./mlruns/{experiment_id}/{run_id}/artifacts/leads_pyfunc -p 5001'. \
    format(experiment_id=experiment_id,run_id=run_id)

print(sagemaker_local_command)

mlflow sagemaker run-local -m ./mlruns/1/a23c5e4b46174c5f989bebda83458003/artifacts/leads_pyfunc -p 5001


In [36]:
# Building a function to test out the locally-build sagemaker container
def query_local_endpoint(input_json):
    response = requests.post('http://localhost:5001/invocations'
                           , headers = {'Content-Type': 'application/json'} 
                           , data=input_json)
    print(response)
    preds = response.json()
    return preds

In [37]:
# Running a query against the local endpoint and examining the output
model_input = leads_x.head(1)
output=query_local_endpoint(model_input.to_json(orient="split"))
print(output)

ConnectionError: HTTPConnectionPool(host='localhost', port=5001): Max retries exceeded with url: /invocations (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000018EFDF8D6A0>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

In [ ]:
########################################################
### Deploy Model to Sagemaker
########################################################

In [ ]:
## Note: this requires a MLflow pyfunc docker container to already exist in sagemaker

import mlflow.sagemaker as mfs


# we pull the run and experiment id's from above to create this mlflow location
model_uri = "mlruns/%s/%s/artifacts/leads_pyfunc" % (experiment_id,run_id)

# The region is chosen, pick whats close to you or your systems!
region = "us-east-1"
# The aws account id can be found in the console
aws_account_id = "XXXXXXX"
# We use these inputs to automatically reference the sagemaker docker container
image_url = aws_account_id \
            + ".dkr.ecr." \
            + region \
            + ".amazonaws.com/mlflow-pyfunc:1.5.0"

# now we specify the role that we setup for sagemaker in the previous step
sagemaker_arn = "arn:aws:iam::"+aws_account_id+":role/AmazonSageMakerFullAccess"


# finally, we pick a name for our endpoint within sagemaker
endpoint_name = "lead-rf-1" 


# with all of the inputs, we run the following to deploy the model it sagemaker
mfs.deploy(app_name=endpoint_name, 
           model_uri=model_uri,
           region_name=region,
           mode="create", #this should change to replace if the endpoint already exists
           execution_role_arn=sagemaker_arn,
           image_url=image_url, 
           instance_type='ml.t2.medium') # smallest/cheapest sagemaker allowed size